# Paice method for evaluation of Stemmer

## Stemming
Stemming of a word essentially means to remove all linguistic inflections present in the word and transform it into a simple *word stem*. This is usually done to improve recall in an IR system (think search engine) and also for removing noises while constructing statistical models like *Bag of Words*. For example, the words *walking, walked,* and *walks* are all variations of the verb *walk*. In the context of a search engine, it is desirable to fetch all the documents containing these terms when the input query is *walk*.

Stemming is usually done by stripping off suffixes and prefixes from words. If we are to consider a simple stemmer which strips only the suffixes *-ing, -ed,* and *-s*, then the words can be stemmed as follows:

* walking -> walk
* walked -> walk
* walks -> walk
* walk -> walk

The above example can be a bit misleading because stemming does not need to produce a grammatically correct word. Consider the words *tasted* and *tasting*. When stemmed by stripping the suffixes described above, these words become:

* tasted -> tast
* tasting -> tast

This is not necessarily incorrect. All that is required of a stemmer is that it conflate the related inflections to a single stem even if the stem is not a grammatically valid word. But consider the word *taste* which, when stemmed, remains *taste* i.e. it has different stem than *tasted* and *tasting* even though all three of them are related. This is an error. A *under-stemming* error to be precise.

<img src="images/stem_walk.png"></img>

## Evaluating Stemmers

Suppose you write a stemming scheme and also implement it in a programming language of your choice. How do you then evaluate just how good your stemmer is? That is to say, how do you know how your stemmer fares against any other stemmer? What is needed is a quantifying metric which could assess the goodness of your implementation. There are a few ways one could go about it.

### Error rate calculation

One way of evaluating stemmers would be to manually construct a dataset consisting of a word and its corresponding stem. For instance, one entry in such a dataset would be *`<walking, walk>`*. *Walking* being the word and *walk* being the desired stem. The evaluation is then done by stemming all the words and checking if the stem produced does indeed match up with the stem provided in the dataset. For example, consider the dataset to have these four entries.

1. `<walking, walk>`
2. `<walked, walk>`
3. `<tasting, tast>`
4. `<taste, tast>`

When the stemmer from before is run on the dataset, the results obtained would be:

1. `<walking, walk>`
2. `<walked, walk>`
3. `<tasting, tast>`
4. `<taste, taste>`

Clearly, output of the fourth entry doesn't match up with the one in the dataset. Hence, one word out of four was stemmed incorrectly; the error then would be $$\frac 1 4 = 25\%$$

This method has the advantage that it is easily implemented. One need only provide the list of manually paired word and its stem. This method, however, is conceptually flawed. The testing is done by comparing the output of the stemmer to the correct stem. *But there is no such thing as a correct stem*. A stemmer can very well reduce the word *walking* to the stem *wal* and it would be correct so long as it also reduces other inflections (*walked* and *walk*) to the same.
 

### Over-Stemming vs Under-Stemming calculation

Two very useful metrics to quantify a stemmer with are the over and under stemming errors. Under-stemming is when two related words do not reduce to the same stem. We saw that using the previous stemmer, the word *tasting* conflated to the stem *tast* whereas the word *taste* conflated to the stem *taste*. The two words are related inflections but the stemmer doesn't reduce the two to the same.

Similarly, consider the words *red* and *ring*. These two are totally unrelated words. However, using our stemmer, these two are conflated to the same stem i.e. *r*. This is over-stemming i.e. reducing two unrelated words into a same stem.

### Paice's Method

Counting over-stemming and under-stemming errors seems to be a very useful evaluation scheme for stemmers. After all, this method doesn't need the concept of a correct stem and instead only requires that two related words conflate to a same stem and two unrelated words to different. This is precisely what Paice's method does. To illustrate the concept further, let us first create a toy stemmer.

In [15]:
class Stemmer:
    def __init__(self, suffixes):
        assert isinstance(suffixes, list)
        self.suffixes = suffixes[:]
        
    def stem_word(self, word):
        for suffix in self.suffixes:
            if word.endswith(suffix):
                word = word[:-len(suffix)]
                break
        return word
    
    def stem_words(self, words):
        assert isinstance(words, list)
        return map(self.stem_word, words)

toy_stemmer = Stemmer(["s", "ing", "ed"])
print toy_stemmer.stem_words(["tasting", "tasted", "taste"])
# print toy_stemmer.stem_words(["walk", "walked", "walking"])
# print toy_stemmer.stem_words(["red", "ring"])

['tast', 'tast', 'taste']


It is important to understand that the stemmer above strips one and only one suffix from the given word. So the word _winnings_ will be stemmed to _winning_ i.e. it only strips the suffix _s_ and not the second suffix _ing_. Later, we'll modify this stemmer to repeatedly strip all the suffixes present and compare it to the existing one.

Once we have a functional stemmer, to evaluate it we'll need to count the over-stemming and under-stemming errors. But before we do that, we also need to know just which words are meant to conflate to a same stem and which words to different one. In Paice method, this is done by introducing what is known as a *concept group*, which is essentially a list of words that are meant to conflate to the same word stem. For this example, we use following concept groups.

1. walk, walks, walked, walking
2. taste, tastes, tasted, tasting
3. red, reds
4. ring, ringing

<!--The idea is that whenever you are querying an IR system using a word from one group, the IR should fetch documents that also contains different words from the same group. For instance, suppose you search *George walked* then you might also be interested in documents that contains the text *George was seen walking to his home*. -->

We can extend the notion of *concept groups* further by introducing *concept pairs* and *non-concept pairs*. A *concept pair* is a pair of words which should conflate to the same stem. From group (1) above, we can derive following *concept pairs*.

* `<walk, walks>`
* `<walk, walked>`
* `<walk, walking>`
* `<walks, walked>`
* `<walks, walking>`
* `<walked, walking>`

Notice that the ordering doesn't really matter here, so even though the pair `<walks, walk>` is a valid _concept pair_, it is not included because `<walk, walks>` is already present in the list.

Just how many concept pairs can be obtained from a group with _n_ number of words? With simple combinatorics, the answer is $$n\choose2$$

Again, if we have the number of _concept pairs_ from one group, then the number of _concept pairs_ across all the groups becomes $$\sum_{g}{n_g\choose2}$$

Where $n_g$ is the number of words in group _g_. This is also known as the _Global Desired Merge Total_ or **GDMT**. This can also be rewritten as:

$$GDMT = \sum_{g}{n_g\choose2} = \sum_{g}{\frac{n_g(n_g-1)}{2}} $$

Moving on to the *non-concept pairs*, it is merely the pairs of words that are not meant to conflate together to a same stem. For instance, using group (3) from above, we can derive the following _non-concept pairs_.

* `<red, walk>`
* `<red, walks>`
* `<red, walked>`
* `<red, walking>`
* `<red, taste>`
* `<red, tastes>`
* `<red, tasted>`
* `<red, tasting>`
* `<red, ring>`
* `<red, rings>`

To get the number of non-concept pairs from one group, consider the following approach. Suppose there are $n_g$ number of words in a group and suppose that the total number of words across all groups is $W$. First off, we choose one word from the said group. This can be done in $n_g$ number of ways. After that, we choose any word from the rest of the group, which can be done in $W-n_g$ ways. Again, since the ordering doesn't really matter, we divide the whole result by 2. Mathematically, this gives the number of *non-concept pairs* from a group as:

$$\frac{n_g(W-n_g)}{2}$$

Once gain, the total number of *non-concept pairs* can be found by summing the result above across all *concept groups*. This is also called the *Global Desired Non-Merge Total* or **GDNT**.

$$ GDNT = \sum_{g}{\frac{n_g(W-n_g)}{2}}$$

Let us now calculate the metrics $GDMT$ and $GDNT$ for the concept group described above.

In [16]:
concept_groups = [
    ['walk', 'walks', 'walked', 'walking'],
    ['taste', 'tastes', 'tasted', 'tasting'],
    ['red', 'reds'],
    ['ring', 'ringing']
]

def GDMT(cg):
    gdmt = 0
    for g in cg:
        ng = len(g)
        gdmt += 0.5 * ng * (ng-1)
    return gdmt

def GDNT(cg):
    gdnt = 0
    W = sum([len(g) for g in cg])
    for g in cg:
        ng = len(g)
        gdnt += 0.5 * ng * (W-ng)
    return gdnt

print "GDMT = %d" % GDMT(concept_groups)
print "GDNT = %d" % GDNT(concept_groups)

GDMT = 14
GDNT = 52


This shows that there are 14 _concept pairs_ i.e. words that should conflate to the same stem and 52 _non-concept pairs_ i.e. words that shouldn't conflate to the same stem.

To evaluate the performance of our stemmer, we must count how many of the *concept pairs* were unmerged i.e. did not conflate to the same stem and how many of the *non-concept pairs* were wrongly merged i.e. conflated to the same stem. This is better illustrated with an example.

Consider _concept group_ (2) which has the following words:

* taste, tastes, tasted, tasting

When stemmed, these words give the following stems.

In [17]:
group_2 = concept_groups[1]
print zip(group_2, toy_stemmer.stem_words(group_2))

[('taste', 'taste'), ('tastes', 'taste'), ('tasted', 'tast'), ('tasting', 'tast')]


### Unmerged Concept Pairs and Understemming Error
Simple observation tells us that the unmerged _concept pairs_ for this _concept group_ are:

* `<taste, tasted>`
* `<taste, tasting>`
* `<tastes, tasted>`
* `<tastes, tasting>`

Concretely, _taste_ stemmed to _taste_ while _tasted_ stemmed to _tast_ and so on for the rest of the three pairs. To count these unmerged pairs for a _concept group_, following algorithm could be used:

1. Determine all unique stems for a concept group.
2. Count the number of words associated with each stem.
3. For each unique stem, calculate $0.5 * u_i(n_g-u_i)$. Where $u_i$ is the number of words that reduce to that particular stem and $n_g$ is the number of words in that group.
4. Sum up (3) for all unique stems.

The concept group above gives two distinct stems for the constituent words. These distinct stems and their instances are:

* taste -> 2 times (taste and tastes)
* tast  -> 2 times (tasting and tasted)

The concept group (3) has 4 words in it. According to the algorithm above, the total unmerged pairs in this group then becomes

- For _taste_
  - $0.5 * 2 * (4-2) = 2$
- For _tasting_
  - $0.5 * 2 * (4-2) = 2$

i.e. 4 unmerged pairs in total, which is the same as the number of unmerged pairs listed above. When the number of unmerged pairs is calculated for all of the concept groups and summed up, this gives the total _concept pairs_ that were unmerged. This value is called _Global Unachieved Merge Total_ or **GUMT**.

In [29]:
from collections import Counter
def GUMT(cg, debug=False):
    gumt = 0
    for g in cg:
        stems = toy_stemmer.stem_words(g)
        ng = len(g)
        unique_stems = Counter(stems)
        if debug:
            print "For group " + repr(g) + "\n" + "Unique stems with their instances are: " \
            + repr(dict(unique_stems))
        umt = 0
        for unique_stem, count in unique_stems.items():
            umt += 0.5 * count * (ng-count)
        gumt += umt
        if debug:
            print "Unmerged concept pairs in this group = %d\n" % umt

    return gumt

print "GUMT = %d" % GUMT(concept_groups, True)

For group ['walk', 'walks', 'walked', 'walking']
Unique stems with their instances are: {'walk': 4}
Unmerged concept pairs in this group = 0

For group ['taste', 'tastes', 'tasted', 'tasting']
Unique stems with their instances are: {'taste': 2, 'tast': 2}
Unmerged concept pairs in this group = 4

For group ['red', 'reds']
Unique stems with their instances are: {'r': 1, 'red': 1}
Unmerged concept pairs in this group = 1

For group ['ring', 'ringing']
Unique stems with their instances are: {'ring': 1, 'r': 1}
Unmerged concept pairs in this group = 1

GUMT = 6


Once we have the $GUMT$, we can then find out the _understemming error_. This is simply given by the total number of _unmerged concept pairs_ (or $GUMT$) divided by total number of _concept pairs_ (or $GDMT$). This understemming error is also called the _Understemming Index_ or __UI__.

In [35]:
def UI(cg):
    return GUMT(cg)/GDMT(cg)

print "Understemming Error = %.2f%%" % (UI(concept_groups)*100)

Understemming Error = 42.86%


### Wrongly merged concept pairs and Overstemming Error

